In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader,TensorDataset
from gensim.models import Word2Vec
from models import CNNClassifier, RNNClassifier, GRUClassifier, LSTMClassifier, TClassifier
from config_dataset import custom_dataset
from accuracy import classifier_accuracy

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
sequence_length = 20
batch_size = 32
embedding_dim = 300
hidden_dim = 8

In [4]:
train_loader, val_loader, embedding_dim, embedding_matrix, word2vec, idx2word, word2idx, vocab_size, style0_test, style1_test, style2_test = custom_dataset('text_corpus/divina_commedia.txt',
                                                                                                                                                        'text_corpus/uno_nessuno_e_i_malavoglia.txt',
                                                                                                                                                        'text_corpus/lo_cunto_de_li_cunti.txt',
                                                                                                                                                        sequence_length,
                                                                                                                                                        embedding_dim,
                                                                                                                                                        batch_size,
                                                                                                                                                        0.8)
print('len train loader: ', len(train_loader))
print('len val loader: ', len(val_loader))
print('total number of training samples: ', len(train_loader.dataset))
print('total number of validation samples: ', len(val_loader.dataset))
print('vocab size: ', vocab_size)

len train loader:  633
len val loader:  80
total number of training samples:  20246
total number of validation samples:  2529
vocab size:  26606


# CNN Classifier

In [5]:
cnn_classifier = CNNClassifier(embedding_matrix, 3, 3, [3,3,3])

cnn_classifier.load_state_dict(torch.load('pretrained/cnn_classifier.pth'))

classifier_params = sum(p.numel() for p in cnn_classifier.parameters() if p.requires_grad)
print('# Parameters: ', classifier_params)

# Parameters:  8139


In [6]:
classifier_accuracy(cnn_classifier, style0_test, style1_test, style2_test, 'CNN')

Accuracy predicting Dante:  0.9813176007866273
Accuracy predicting Italian:  0.992133726647001
Accuracy predicting Neapolitan:  0.998
Overall  CNN  Accuracy:  0.9904837758112094


In [7]:
for i ,(data, label) in enumerate(val_loader):
    if i == 0:
        prova = data[0]
        labels = label[0]
    else: 
        break

input_sentence = [idx2word[prova[i].item()] for i in range(prova.shape[0])]

prova = prova.view(1,prova.shape[0])

with torch.no_grad():
    pred_label = cnn_classifier(prova)
    
pred_label = torch.softmax(pred_label, dim = -1)


if labels == 0.0:
    stile = 'Dante'
elif labels == 1.0:
    stile = 'Italiano'
elif labels == 2.0:
    stile = 'Napoletano'


pred_style = torch.argmax(pred_label).item()
print('Real Style: ', labels.item(), ', ', stile)
print('Predicted Style: ', pred_style, 'with probability: ', pred_label[0][0][pred_style].item()*100, '%')
print("Input sequence: \n", ' '.join(input_sentence))

Real Style:  1 ,  Italiano
Predicted Style:  1 with probability:  57.61128067970276 %
Input sequence: 
 peggio dei lupi colle pistole e le carabine ma rubare ai ladri non è stato mai peccato lo diceva anche


# RNN Classifier

In [8]:
rnn_classifier = RNNClassifier(embedding_matrix, hidden_dim, 1)

rnn_classifier.load_state_dict(torch.load('pretrained/rnn_classifier.pth'))

classifier_params = sum(p.numel() for p in rnn_classifier.parameters() if p.requires_grad)
print('# Parameters: ', classifier_params)

# Parameters:  3107


In [9]:
classifier_accuracy(rnn_classifier, style0_test, style1_test, style2_test, 'RNN')

Accuracy predicting Dante:  0.9695181907571289
Accuracy predicting Italian:  0.976401179941003
Accuracy predicting Neapolitan:  0.962
Overall  RNN  Accuracy:  0.9693064568993771


In [10]:
for i ,(data, label) in enumerate(val_loader):
    if i == 0:
        prova = data[0]
        labels = label[0]
    else: 
        break

input_sentence = [idx2word[prova[i].item()] for i in range(prova.shape[0])]

prova = prova.view(1,prova.shape[0])

with torch.no_grad():
    pred_label = rnn_classifier(prova)
    
pred_label = torch.softmax(pred_label, dim = -1)


if labels == 0.0:
    stile = 'Dante'
elif labels == 1.0:
    stile = 'Italiano'
elif labels == 2.0:
    stile = 'Napoletano'


pred_style = torch.argmax(pred_label).item()
print('Real Style: ', labels.item(), ', ', stile)
print('Predicted Style: ', pred_style, 'with probability: ', pred_label[0][0][pred_style].item()*100, '%')
print("Input sequence: \n", ' '.join(input_sentence))

Real Style:  0 ,  Dante
Predicted Style:  0 with probability:  98.8381028175354 %
Input sequence: 
 cristiani in ferma fede quel d i passuri e quel d i passi piedi chè l una de lo nferno


# GRU Classifier

In [11]:
gru_classifier = GRUClassifier(embedding_matrix, hidden_dim, 1)

gru_classifier.load_state_dict(torch.load('pretrained/gru_classifier.pth'))

classifier_params = sum(p.numel() for p in gru_classifier.parameters() if p.requires_grad)
print('# Parameters: ', classifier_params)

# Parameters:  8067


In [12]:
classifier_accuracy(gru_classifier, style0_test, style1_test, style2_test, 'GRU')

Accuracy predicting Dante:  0.9626352015732547
Accuracy predicting Italian:  0.9911504424778761
Accuracy predicting Neapolitan:  0.99
Overall  GRU  Accuracy:  0.981261881350377


In [13]:
for i ,(data, label) in enumerate(val_loader):
    if i == 0:
        prova = data[0]
        labels = label[0]
    else: 
        break

input_sentence = [idx2word[prova[i].item()] for i in range(prova.shape[0])]

prova = prova.view(1,prova.shape[0])

with torch.no_grad():
    pred_label = gru_classifier(prova)
    
pred_label = torch.softmax(pred_label, dim = -1)


if labels == 0.0:
    stile = 'Dante'
elif labels == 1.0:
    stile = 'Italiano'
elif labels == 2.0:
    stile = 'Napoletano'


pred_style = torch.argmax(pred_label).item()
print('Real Style: ', labels.item(), ', ', stile)
print('Predicted Style: ', pred_style, 'with probability: ', pred_label[0][0][pred_style].item()*100, '%')
print("Input sequence: \n", ' '.join(input_sentence))

Real Style:  0 ,  Dante
Predicted Style:  0 with probability:  99.92274045944214 %
Input sequence: 
 tutta l anima tolta così nel fiammeggiar del folgor santo a ch io mi volsi conobbi la voglia in lui


# LSTM Classifier

In [14]:
lstm_classifier = LSTMClassifier(embedding_matrix, hidden_dim, 1)

lstm_classifier.load_state_dict(torch.load('pretrained/lstm_classifier.pth'))

classifier_params = sum(p.numel() for p in lstm_classifier.parameters() if p.requires_grad)
print('# Parameters: ', classifier_params)

# Parameters:  10547


In [15]:
classifier_accuracy(lstm_classifier, style0_test, style1_test, style2_test, 'LSTM')

Accuracy predicting Dante:  0.9793510324483776
Accuracy predicting Italian:  0.9941002949852508
Accuracy predicting Neapolitan:  0.994
Overall  LSTM  Accuracy:  0.9891504424778761


In [16]:
for i ,(data, label) in enumerate(val_loader):
    if i == 0:
        prova = data[0]
        labels = label[0]
    else: 
        break

input_sentence = [idx2word[prova[i].item()] for i in range(prova.shape[0])]

prova = prova.view(1,prova.shape[0])

with torch.no_grad():
    pred_label = lstm_classifier(prova)
    
pred_label = torch.softmax(pred_label, dim = -1)


if labels == 0.0:
    stile = 'Dante'
elif labels == 1.0:
    stile = 'Italiano'
elif labels == 2.0:
    stile = 'Napoletano'


pred_style = torch.argmax(pred_label).item()
print('Real Style: ', labels.item(), ', ', stile)
print('Predicted Style: ', pred_style, 'with probability: ', pred_label[0][0][pred_style].item()*100, '%')
print("Input sequence: \n", ' '.join(input_sentence))

Real Style:  0 ,  Dante
Predicted Style:  0 with probability:  99.90511536598206 %
Input sequence: 
 mortali è tanto blanda che giù non basta buon cominciamento dal nascer de la quercia al far la ghianda pier


# Transformer Classifier

In [17]:
t_classifier = TClassifier(embedding_matrix)

t_classifier.load_state_dict(torch.load('pretrained/t_classifier.pth'))

classifier_params = sum(p.numel() for p in t_classifier.parameters() if p.requires_grad)
print('# Parameters: ', classifier_params)

# Parameters:  3188599


In [18]:
classifier_accuracy(t_classifier, style0_test, style1_test, style2_test, 'Transformer')

Accuracy predicting Dante:  0.9911504424778761
Accuracy predicting Italian:  0.9941002949852508
Accuracy predicting Neapolitan:  0.998
Overall  Transformer  Accuracy:  0.994416912487709


In [19]:
for i ,(data, label) in enumerate(val_loader):
    if i == 0:
        prova = data[0]
        labels = label[0]
    else: 
        break

input_sentence = [idx2word[prova[i].item()] for i in range(prova.shape[0])]

prova = prova.view(1,prova.shape[0])

with torch.no_grad():
    pred_label = t_classifier(prova)
    
pred_label = torch.softmax(pred_label, dim = -1)


if labels == 0.0:
    stile = 'Dante'
elif labels == 1.0:
    stile = 'Italiano'
elif labels == 2.0:
    stile = 'Napoletano'


pred_style = torch.argmax(pred_label).item()
print('Real Style: ', labels.item(), ', ', stile)
print('Predicted Style: ', pred_style, 'with probability: ', pred_label[0][0][pred_style].item()*100, '%')
print("Input sequence: \n", ' '.join(input_sentence))

Real Style:  1 ,  Italiano
Predicted Style:  1 with probability:  99.99983310699463 %
Input sequence: 
 no santo dio ntoni dacchè la santuzza gli faceva degli sgarbi bisognava che pensasse come pagare il pane che gli
